In [ ]:
from utils import *
import pickle
import datetime
import scipy
from scipy.optimize import minimize

In [ ]:
purples = ['#f7fcfd','#e0ecf4','#bfd3e6','#9ebcda','#8c96c6','#8c6bb1','#88419d','#810f7c','#4d004b']
oranges = ['#ffffe5','#fff7bc','#fee391','#fec44f','#fe9929','#ec7014','#cc4c02','#993404','#662506']

In [ ]:
params=Parameters(B=0., u=0., Tl=1.3, Tr=1., gamma=.1, kerntype='RTD', dband=1e4, countingleads=[0,2])

In [ ]:
def power(Vs,params):
    vg,vb = Vs
    system = anderson(vg,vb,params)
    system.solve()
    return -vb*np.sum(system.current[[0,2]])

def eff(Vs,params):
    vg,vb = Vs
    system = anderson(vg,vb,params)
    system.solve()
    p = -vb*np.sum(system.current[[0,2]])
    q = np.sum(system.heat_current[[0,2]])
    return p/q


cons = ({'type': 'ineq', 'fun': lambda x:  power(x,params)-1e-5})

In [ ]:
def main(Vg, Vb, params):
    with ProcessPoolExecutor() as executor:
        results = np.array(list(tqdm(executor.map(solve_bias_gate_p,Vg,Vb,params),total=len(Vg))),dtype=object)
    return results

In [ ]:
def main_srl(Vg,Vb,Gamma):
    with ProcessPoolExecutor() as executor:
        results = np.array(list(tqdm(executor.map(SRL_exact,Gamma,Vg,Vb,[1.3]*len(Vg),[1]*len(Vg)),total=len(Vg))))
    return results

In [ ]:
U = [100,0]
Gamma = np.linspace(0.01,0.4,20)
T,dT=1.,0.3

In [ ]:

Pmax_rtd = [[] for i in range(len(U))]
Pmax_pauli = [[] for i in range(len(U))]
Pmax_lb = []

In [ ]:
if __name__ == '__main__':
    params=Parameters(B=0., u=0, Tl=T+dT, Tr=T, gamma=.1, kerntype='RTD', dband=1e4, countingleads=[0,2])
    for i,u in enumerate(U):
        params.u = u
        for j,gamma in enumerate(tqdm(Gamma, desc='gamma')):
            params.gamma = gamma
            if gamma == Gamma.min():
                x0 = [-1,-1]
            else:
                x0 = Pmax_rtd[i][j-1][1].x
            res = gamma, minimize( lambda x: -power(x,params), x0, tol= 1e-8, options={'maxiter': 200})
            Pmax_rtd[i].append(res)
            #print(gamma, res[1].x, -res[1].fun)

In [ ]:
if __name__ == '__main__':
    params=Parameters(B=0., u=0, Tl=T+dT, Tr=T, gamma=.1, kerntype='Pauli', dband=1e4, countingleads=[0,2])
    for i,u in tqdm(enumerate(U), desc='U', total=len(U)):
        params.u = u
        for j,gamma in enumerate(tqdm(Gamma, desc='gamma')):
            params.gamma = gamma
            if gamma == Gamma.min():
                x0 = [-1,-1]
            else:
                x0 = Pmax_pauli[i][j-1][1].x
            res = gamma, minimize( lambda x: -power(x,params), x0, tol= 1e-8, options={'maxiter': 1000})
            Pmax_pauli[i].append(res)
            #print(gamma, res[1].x, -res[1].fun)

In [ ]:
if __name__ == '__main__':
    for gamma in tqdm(Gamma, desc='gamma'):
        if gamma == Gamma.min():
            x0 = [-1,-1]
        else:
            x0 = Pmax_lb[-1][1].x
        res = gamma, minimize( lambda x: x[1]*SRL_exact(gamma,x[0],x[1],T+dT,T,dband=1e2)[0], x0, tol = 1e-8, options={'maxiter': 1000})
        Pmax_lb.append(res)
        #print(gamma, res[1].x, -res[1].fun)

In [ ]:
data = {'Pmax_rtd':Pmax_rtd, 'Pmax_pauli':Pmax_pauli, 'Pmax_lb':Pmax_lb, 'U':U, 'Gamma':Gamma}

In [ ]:

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# pickle data
with open(timestamp+'pmax_outside_gammasweep.pkl', 'wb') as f:
    pickle.dump(data, f)# pickle Pmax

In [ ]:
Vg = [[p[1].x[0] for p in P] for P in Pmax_rtd]
Vb = [[p[1].x[1] for p in P] for P in Pmax_rtd]
Vg_pauli = [[p[1].x[0] for p in P] for P in Pmax_pauli]
Vb_pauli = [[p[1].x[1] for p in P] for P in Pmax_pauli]
Vg_lb = [p[1].x[0] for p in Pmax_lb]
Vb_lb = [p[1].x[1] for p in Pmax_lb]

params=[Parameters(B=0., u=0, Tl=T+dT, Tr=T, gamma=gamma, kerntype='RTDnoise', dband=1e4, countingleads=[0,2]) for gamma in Gamma]



In [ ]:
cols=[oranges[5],purples[5],oranges[3],purples[3],'k']
for i in range(2):
    plt.plot(Vg[i],Vb[i],color=cols[2*i], marker='x',label='Second order U='+str(data['U'][i]))
for i in range(2):
    plt.plot(Vg_pauli[i],Vb_pauli[i],color=cols[2*i+1], marker='x',label='First order U='+str(data['U'][i]))
plt.plot(Vg_lb,Vb_lb,'k--',label='Exact LB U=0')
plt.legend(loc='lower right')
plt.title('Position max. power point')
plt.xlabel(r'$V_g/T$')
plt.ylabel(r'$V_b/T$')
plt.savefig('maxpp.pdf',bbox_inches='tight')
plt.show()

In [ ]:
results_rtd = []
for i,u in tqdm(enumerate(U), desc='U', total=len(U)):
    for p in params:
        p.u = u
        p.kerntype='RTDnoise'
    res = main(Vg[i],Vb[i],params)
    results_rtd.append(res)

In [ ]:
results_pauli = []
for i,u in tqdm(enumerate(U), desc='U', total=len(U)):
    for p in params:
        p.u = u
        p.kerntype = 'Pauli'
    res = main(Vg_pauli[i],Vb_pauli[i],params)
    results_pauli.append(res)

In [ ]:
results_exact = main_srl(Vg_lb,Vb_lb,Gamma)

In [ ]:
I_rtd = np.array([[r[0][0] for r in R] for R in results_rtd])
S_rtd = np.array([[r[0][1] for r in R] for R in results_rtd])
F_rtd = S_rtd/I_rtd
Q_rtd = np.array([[r[2][0]+r[2][2] for r in R] for R in results_rtd])
P_rtd = -I_rtd*np.array(Vb)
nc_rtd= 1 - (T/(T+dT))
n_rtd = P_rtd/Q_rtd
TUR_rtd = -S_rtd/I_rtd*np.array(Vb)/1*(nc_rtd-n_rtd)/n_rtd
TURS_rtd =-2*I_rtd/Vb*1/(nc_rtd-n_rtd)*n_rtd
TURI_rtd =-S_rtd/2*Vb/1*(nc_rtd-n_rtd)/n_rtd
TURn_rtd =nc_rtd/((-2*I_rtd*1)/S_rtd/Vb+1)



I_exact = 2*np.array([r[0] for r in results_exact])
S_exact = 2*np.array([r[1] for r in results_exact])
F_exact = S_exact/I_exact
Q_exact = 2*np.array([r[2] for r in results_exact])
P_exact = -I_exact*Vb_lb
nc_exact = 1 - (T/(T+dT))
n_exact = P_exact/Q_exact
TUR_exact = -S_exact/I_exact*Vb_lb/1*(nc_exact-n_exact)/n_exact

In [ ]:
I_pauli = np.array([[r[0][0] for r in R] for R in results_pauli])
S_pauli = np.array([[r[0][1] for r in R] for R in results_pauli])
F_pauli = S_pauli/I_pauli
Q_pauli = np.array([[r[2][0]+r[2][2] for r in R] for R in results_pauli])
P_pauli = -I_pauli*np.array(Vb_pauli[0])
nc_pauli= 1 - (T/(T+dT))
n_pauli = P_pauli/Q_pauli
TUR_pauli= -S_pauli/I_pauli*np.array(Vb_pauli[0])/1*(nc_pauli-n_pauli)/n_pauli

In [ ]:
res_plt = [[I_rtd,I_pauli,I_exact],[S_rtd,S_pauli,S_exact],[F_rtd, F_pauli, F_exact],[P_rtd,P_pauli,P_exact],[Q_rtd,Q_pauli,Q_exact],[n_rtd/nc_rtd,n_pauli/nc_pauli,n_exact/nc_exact]]
# pickle res_plt
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
with open(timestamp+'res_plt.pkl', 'wb') as f:
    pickle.dump(res_plt, f)



In [ ]:
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
# plot I, S, Q, P, n, TUR vs gamma in 6 subplots
cols=[oranges[5],purples[5],oranges[3],purples[3],'k']
labs = [r'Current $I/\Gamma$',r'Noise $S/\Gamma$',r'Fano factor $F=S/I$',r'Power $P/(\Gamma \cdot T)$',r'Heat current $Q/\Gamma$',r'Efficiency $n/n_c$']
label_pos = [[0.02,0.98],[0.02,0.98],[0.02,0.98],[0.02,0.98],[0.02,0.98],[0.02,0.98]]
fig, axs = plt.subplots(2,3,figsize=(12,6),sharex=True)
for i in [0,1,4]:
    ax=axs.flatten()[i]
    for j,u in enumerate(U):
        ax.plot(Gamma, res_plt[i][0][j].real/Gamma, label='Second order U='+str(u),color=cols[2*j])
        ax.plot(Gamma, res_plt[i][1][j].real/Gamma, label='First order U='+str(u),color=cols[2*j+1])
        #print(j,u)
    ax.plot(Gamma, res_plt[i][2].real/Gamma,'k--', label='Exact LB U=0')
    ax.text(label_pos[i][0],label_pos[i][1],f'({chr(97+i)}) '+labs[i],fontsize='large', horizontalalignment='left',verticalalignment='top',transform=ax.transAxes)
    ax.set_xmargin(0)
    ax.grid(True,which='both',axis='both',linestyle='--',linewidth=0.5)
    #ax.legend()
for i in [3]:
    ax=axs.flatten()[i]
    for j,u in enumerate(U):
        ax.plot(Gamma, res_plt[i][0][j].real/Gamma/T, label='Second order U='+str(u),color=cols[2*j])
        ax.plot(Gamma, res_plt[i][1][j].real/Gamma/T, label='First order U='+str(u),color=cols[2*j+1])
        #print(j,u)
    ax.plot(Gamma, res_plt[i][2].real/Gamma/T,'k--', label='Exact LB U=0')
    ax.text(label_pos[i][0],label_pos[i][1],f'({chr(97+i)}) '+labs[i],fontsize='large', horizontalalignment='left',verticalalignment='top',transform=ax.transAxes)
    ax.set_xmargin(0)
    ax.grid(True,which='both',axis='both',linestyle='--',linewidth=0.5)
    #ax.legend()
for i in [2,5]:
    ax=axs.flatten()[i]
    for j,u in enumerate(U):
        ax.plot(Gamma, res_plt[i][0][j].real, label='Second order U='+str(u),color=cols[2*j])
        ax.plot(Gamma, res_plt[i][1][j].real, label='First order U='+str(u),color=cols[2*j+1])
        #print(j,u)
    ax.plot(Gamma, res_plt[i][2].real,'k--', label='Exact LB U=0')
    ax.text(label_pos[i][0],label_pos[i][1],f'({chr(97+i)}) '+labs[i],fontsize='large', horizontalalignment='left',verticalalignment='top',transform=ax.transAxes)
    ax.set_xmargin(0)
    ax.grid(True,which='both',axis='both',linestyle='--',linewidth=0.5)
    #ax.legend()
axs[1,2].set_ylim(0,1)
for ax in axs[1,:]:
    ax.set_xlabel('$\Gamma/T$')
for ax in axs.flatten()[:-1]:
    #ax.set_ymargin(0)
    ax.set_xlim(0.01,0.4)
    ax.set_ymargin(0.15)
#fig.suptitle('Gamma sweepkkat max power point of RTD\nU from 0 to 100, Tl=1.3,Tr=1\nRTD: orange, Pauli: purple, Exact: black')

#axs[0,2].set_ylim(5.8,res_plt[2][0][0].real.max())

# axs[0,0].plot(Gamma, TURI_rtd[0,:].real,':',color=oranges[3])
# axs[0,0].plot(Gamma, TURI_rtd[1,:].real,':',color=oranges[6])
# axs[0,1].plot(Gamma, TURS_rtd[0,:].real,':',color=oranges[3])
# axs[0,1].plot(Gamma, TURS_rtd[1,:].real,':',color=oranges[6])
# axs[1,2].plot(Gamma, TURn_rtd[0,:].real/nc_rtd,':',color=oranges[3])
# axs[1,2].plot(Gamma, TURn_rtd[1,:].real/nc_rtd,':',color=oranges[6])

handles, labels = axs.flatten()[0].get_legend_handles_labels()
plt.figlegend(handles, labels, loc='upper center', ncol=3)


#plt.figlegend(['Second order U=100', 'First order U=100', 'Second order U=0', 'First order U=0', 'Exact LB'],loc='upper center',ncol=3)

plt.subplots_adjust(hspace=0.05)
plt.savefig('fig3.pdf',bbox_inches='tight')

plt.show()